# Training
Seems like we're ready for training!

In [1]:
import sys, os, datetime
sys.path.append('/home/andrea/AI/ispr_yolo/data')
sys.path.append('/home/andrea/AI/ispr_yolo/model')

from yolo import YOLOv3
from lossfunction import YoloLoss
from DataPreprocessing import create_dataset
import tensorflow as tf

#### Creating the Dataset

In [9]:
img_dir = '/home/andrea/AI/ispr_yolo/data/dataset_bdd/images/100k/train'+ '/*.jpg'
train_ds = create_dataset(img_dir, batch = 2)

#### Compiling the model

In [3]:
yolo = YOLOv3(size = 1280, training = True)

In [4]:
# Clear any logs from previous runs
!rm -rf ./logs/ 

In [5]:
yolo_loss = YoloLoss(10)
yolo.compile(loss=[yolo_loss]*3, optimizer='nadam')

In [6]:
%load_ext tensorboard
logdir = os.path.join("logs/fit", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [ ]:
yolo.fit(train_ds,
         epochs = 100,
         callbacks = [tensorboard_callback])

Epoch 1/100
    1/34932 [..............................] - ETA: 43:54 - loss: 337811.4375 - tf_op_layer_Reshape_2_loss: 70525.7812 - tf_op_layer_Reshape_1_loss: 75302.6484 - tf_op_layer_Reshape_loss: 191983.0312WARNING:tensorflow:From /home/andrea/anaconda2/envs/ai/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
 3711/34932 [==>...........................] - ETA: 10:26:04 - loss: 139003.4688 - tf_op_layer_Reshape_2_loss: 25089.8926 - tf_op_layer_Reshape_1_loss: 21325.8203 - tf_op_layer_Reshape_loss: 92587.7422

In [7]:
%tensorboard --logdir logs/fit